<a href="https://colab.research.google.com/github/mtsyr3/Many-in-Few/blob/master/nb/Qwen2.5_(3B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Load up `Qwen 2.5 3B Instruct`, and set parameters

In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-13 09:40:56 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-13 09:40:56 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.6.2: Fast Qwen2 patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunke

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 06-13 09:41:50 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 06-13 09:41:50 [model_runner.py:1140] Model loading took 2.4392 GiB and 11.724890 seconds
INFO 06-13 09:41:55 [worker.py:287] Memory profiling takes 3.81 seconds
INFO 06-13 09:41:55 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.50) = 7.30GiB
INFO 06-13 09:41:55 [worker.py:287] model weights take 2.44GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 3.79GiB.
INFO 06-13 09:41:55 [executor_base.py:112] # cuda blocks: 6894, # CPU blocks: 0
INFO 06-13 09:41:55 [executor_base.py:117] Maximum concurrency for 1024 tokens per request: 107.72x
INFO 06-13 09:41:55 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If ou

Capturing CUDA graph shapes:   0%|          | 0/27 [00:00<?, ?it/s]

INFO 06-13 09:42:44 [model_runner.py:1592] Graph capturing finished in 49 secs, took 0.56 GiB
INFO 06-13 09:42:44 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 54.14 seconds
Unsloth: Just some info: will skip parsing ['q_norm', 'pre_feedforward_layernorm', 'k_norm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['q_norm', 'pre_feedforward_layernorm', 'k_norm', 'post_feedforward_layernorm']


Unsloth 2025.6.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [4]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
# System prompt
SYSTEM_PROMPT = """
You are a cryptic crossword solving AI. Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def get_crossword_questions(csv_path: str):
    print(f"Loading dataset from {csv_path}...")
    df = pd.read_csv(csv_path)
    print(f"Original dataset size: {len(df)}")

    # --- DATASET VALIDATION ---
    required_columns = ['clue_lower', 'answer_lower', 'enumerator', 'enum_type']
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        raise ValueError(f"CSV is missing required columns: {missing_cols}")
    print("✅ All required columns present.")

    df = df.dropna(subset=required_columns)
    print(f"After cleaning rows with missing values: {len(df)}")

    def format_row(row):
        return {
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': f"{row['clue_lower']}"}
            ],
            'answer': str(row['answer_lower']),
            'enumerator': str(row['enumerator']),
            'enum_type': str(row['enum_type'])
        }

    formatted_data = [format_row(row) for _, row in df.iterrows()]

    # --- PROMPT VALIDATION ---
    print("\n--- Sample Formatted Prompt ---")
    print(json.dumps(formatted_data[0], indent=2))
    print("---------------------------------\n")

    full_dataset = Dataset.from_list(formatted_data)
    return full_dataset


In [5]:
import pandas as pd
import json
# Load your dataset
print("Loading and preparing dataset...")
full_dataset = get_crossword_questions('good_data_df_first_200k.csv')
print(f"✅ Full dataset loaded with {len(full_dataset)} examples.")

print("Shuffling dataset...")
shuffled_dataset = full_dataset.shuffle(seed=42) # Use a seed for reproducibility
print("✅ Dataset shuffled.")

Loading and preparing dataset...
Loading dataset from good_data_df_first_200k.csv...
Original dataset size: 199992
✅ All required columns present.
After cleaning rows with missing values: 199992

--- Sample Formatted Prompt ---
{
  "prompt": [
    {
      "role": "system",
      "content": "\nYou are a cryptic crossword solving AI. Respond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n"
    },
    {
      "role": "user",
      "content": "acquisitive chap, as we see it (8)"
    }
  ],
  "answer": "covetous",
  "enumerator": "(8)",
  "enum_type": "single number"
}
---------------------------------

✅ Full dataset loaded with 199992 examples.
Shuffling dataset...
✅ Dataset shuffled.


In [14]:
# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]

    # Key fix: get the single target answer
    target_answer = answer[0] if isinstance(answer, list) else answer

    # Handle empty answers (important for your dataset)
    if not target_answer or not str(target_answer).strip():
        return [0.0] * len(responses)

    # Simple debug print
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{target_answer}",
          f"\nResponse:\n{responses[0][:100]}...", f"\nExtracted:\n{extracted_responses[0]}")

    # Compare all 8 against the single target
    return [2.0 if extracted.lower().replace(' ', '') == target_answer.lower().replace(' ', '')
            else 0.0 for extracted in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs):
    responses = [batch[0]['content'] for batch in completions]
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"

    print("\n" + "="*80)
    print("ALL 8 GENERATIONS:")
    for i, resp in enumerate(responses):
        print(f"\n--- Generation {i+1} ---")
        print(resp)
    print("="*80 + "\n")

    return [0.5 if re.search(pattern, r, re.DOTALL) else 0.0 for r in responses]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]


def parse_enumerator(enum_str: str):
    return [int(n) for n in re.findall(r'\d+', enum_str)]

def word_count_reward_func(completions, enumerator, **kwargs):
    responses = [batch[0]['content'] for batch in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]

    # Get the single enumerator for all generations
    target_enum = enumerator[0] if isinstance(enumerator, list) else enumerator
    expected_lengths = parse_enumerator(str(target_enum))
    expected_word_count = len(expected_lengths)

    rewards = []
    for response in extracted_responses:
        actual_words = response.strip().split()
        actual_word_count = len(actual_words)
        reward = 1.0 if actual_word_count == expected_word_count else 0.0
        rewards.append(reward)

    return rewards

def length_reward_func(completions, enumerator, **kwargs):
    responses = [batch[0]['content'] for batch in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]

    # Get the single enumerator for all generations
    target_enum = enumerator[0] if isinstance(enumerator, list) else enumerator
    expected_lengths = parse_enumerator(str(target_enum))

    rewards = []
    for response in extracted_responses:
        # Get actual word lengths (letters only)
        words = response.strip().split()
        actual_lengths = [len(re.sub(r'[^a-zA-Z]', '', w)) for w in words]

        # Check if lengths match (handles multi-word properly)
        reward = 1.0 if actual_lengths == expected_lengths else 0.0
        rewards.append(reward)

    return rewards




<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [18]:
import os
os.environ["UNSLOTH_ALLOW_BATCH_SIZE_1"] = "1"  # Force Unsloth to respect batch size 1

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = False,  # CHANGE: vLLM can cause batch dimension issues with GRPO
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,  # Keep at 1
    gradient_accumulation_steps = 1,
    num_generations = 8,  # If still errors, try 4
    max_prompt_length = 256,
    max_completion_length = 400,
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none",
    output_dir = "outputs",
)

# Also make dataset size divisible by 8 just in case
dataset_size = (len(shuffled_dataset) // 8) * 8
dataset_train = shuffled_dataset.select(range(min(dataset_size, 20000)))

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [19]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        correctness_reward_func,
        word_count_reward_func,
        length_reward_func,
    ],
    args = training_args,
    train_dataset = dataset_train,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 119,734,272/3,000,000,000 (3.99% trained)



ALL 8 GENERATIONS:

--- Generation 1 ---
<reasoning>
The clue "take notice? no point (6)" follows a common cryptic definition convention. The solution provided is a straightforward one-letter word, which often fits cryptic clues. Let's break down the word "take" into its components - "take" being five letters and "point" being three letters. The cryptic asking for the answer, "no point," implies that "take" is not necessary or significant, and "no point" can fit the six-letter clue easily.
</reasoning>
...
<answer>
no
</answer>

--- Generation 2 ---
<reasoning>
The clue "take notice? no point (6)" suggests a six-letter word that means "no point" or being uninteresting or unexciting. 
</reasoning>
...
<answer>
BUTCHER
</answer>

--- Generation 3 ---
<reasoning>
The clue suggests we need a six-letter word, and it's related to "notice" but with a negative connotation. The phrase "no point" implies that disregarding or ignoring "take notice" is pointless. This hints at a concept of not ca

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / xmlcount_reward_func / mean,rewards / xmlcount_reward_func / std,rewards / soft_format_reward_func / mean,rewards / soft_format_reward_func / std,rewards / strict_format_reward_func / mean,rewards / strict_format_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std,rewards / word_count_reward_func / mean,rewards / word_count_reward_func / std,rewards / length_reward_func / mean,rewards / length_reward_func / std
1,0.000000,1.210125,0.373272,84.625000,53.000000,117.000000,0.000000,84.625000,53.000000,117.000000,0.000042,0.147625,0.112884,0.187500,0.258775,0.000000,0.000000,0.000000,0.000000,0.875000,0.353553,0.000000,0.000000
2,0.000000,1.411625,0.537371,82.000000,70.000000,114.000000,0.000000,82.000000,70.000000,114.000000,0.000040,0.161625,0.076414,0.375000,0.231455,0.000000,0.000000,0.000000,0.000000,0.750000,0.462910,0.125000,0.353553
3,0.000000,1.338500,0.270590,84.500000,71.000000,99.000000,0.000000,84.500000,71.000000,99.000000,0.000034,0.151000,0.053163,0.187500,0.258775,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
4,0.000000,1.555000,0.507610,92.500000,51.000000,141.000000,0.000000,92.500000,51.000000,141.000000,0.000045,0.117500,0.145615,0.312500,0.258775,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.125000,0.353553
5,0.000000,1.712125,0.381787,109.000000,65.000000,172.000000,0.000000,109.000000,65.000000,172.000000,0.000025,0.087125,0.132870,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.125000,0.353553
6,0.000000,1.265750,0.617075,119.000000,62.000000,300.000000,0.125000,93.142860,62.000000,116.000000,0.000037,0.140750,0.107552,0.375000,0.231455,0.000000,0.000000,0.000000,0.000000,0.750000,0.462910,0.000000,0.000000
7,0.000000,1.629375,0.748805,95.750000,71.000000,150.000000,0.000000,95.750000,71.000000,150.000000,0.000250,0.129375,0.127034,0.375000,0.231455,0.000000,0.000000,0.000000,0.000000,0.875000,0.353553,0.250000,0.462910
8,0.000000,0.710500,0.439663,130.375000,83.000000,199.000000,0.000000,130.375000,83.000000,199.000000,0.000061,-0.039500,0.201113,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.462910,0.000000,0.000000
9,0.000000,1.443875,0.710507,76.375000,58.000000,129.000000,0.000000,76.375000,58.000000,129.000000,0.000509,0.131375,0.143359,0.187500,0.258775,0.000000,0.000000,0.000000,0.000000,0.875000,0.353553,0.250000,0.462910
10,0.000000,1.711875,0.188707,81.500000,61.000000,114.000000,0.000000,81.500000,61.000000,114.000000,0.000932,0.149375,0.079190,0.437500,0.176777,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.125000,0.353553


Streaming output truncated to the last 5000 lines.

-------------------- Question:
came to power and filched deliveries (4,4) 
Answer:
took over 
Response:
<reasoning>
The phrase "came to power and filched deliveries" refers to the popular catchphrase "kno... 
Extracted:
KNOP

ALL 8 GENERATIONS:

--- Generation 1 ---
<reasoning>
The clue states that an entity "loved" in an intranet setting, indicating an organization or company. Additionally, the phrase "be resigned, with business finally booming" gives us a pattern where change (resignation) leads to prosperity ("business finally booming").
</reasoning>
<answer>NETFLIX</answer>

--- Generation 2 ---
<reasoning>
The clue is asking for a phrase that can be formed using the letters from "love intranet to be resigned, with business finally booming (1O)". The word length given is 10 letters. 
</reasoning>
<answer>
remonetize
</answer>

--- Generation 3 ---
<reasoning>
The clue states "love intranet to be resigned, with business finally boo

TrainOutput(global_step=250, training_loss=0.003534370747036519, metrics={'train_runtime': 3294.325, 'train_samples_per_second': 0.607, 'train_steps_per_second': 0.076, 'total_flos': 0.0, 'train_loss': 0.003534370747036519})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
